# Setting up the Fast Withdrawal contract:
- This notebook allows deploying contracts for fast withdrawal interactions and running a basic scenario by creating a withdrawal on the Etherlink side and making fast payouts on the Tezos side.

In [1]:
from docs.scenarios.setup import *

web3, etherlink_account, tezos_account = setup()

Setup:
- Tezos account: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`, balance: `299.525507 ꜩ`
- Etherlink account: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`, balance: `609.529149787 ꜩ`


## Deploy FastWithdrawal:

In [2]:
from scripts.tezos import deploy_fast_withdrawal
from scripts.helpers.contracts.fast_withdrawal import FastWithdrawal
from scripts.etherlink import xtz_fast_withdraw

if FAST_WITHDRAWAL_CONTRACT:
    fast_withdrawal = FastWithdrawal.from_address(tezos_account, FAST_WITHDRAWAL_CONTRACT)
    click.echo('Loaded Fast Withdrawal contract, address: ' + wrap(accent(fast_withdrawal.address)))
else:
    fast_withdrawal = deploy_fast_withdrawal.callback(
        xtz_ticketer_address=XTZ_TICKETER_ADDRESS,
        smart_rollup_address=SMART_ROLLUP_ADDRESS,
        expiration_seconds=90*60,
        tezos_private_key=TEZOS_PRIVATE_KEY,
        tezos_rpc_url=TEZOS_RPC_URL,
        skip_confirm=True,
        silent=False,
    )

Loaded Fast Withdrawal contract, address: `KT1TbrmAD4WRVqakbqrHDZ7gUwMdjocXc7Cw`


## Making Fast Withdrawal on Etherlink side:

In [3]:
discounted_amount=9000
full_amount=9119

tx_hash = xtz_fast_withdraw.callback(
    target=get_address(tezos_account),
    fast_withdrawal_contract=fast_withdrawal.address,
    amount=full_amount * 10**12,
    discounted_amount=discounted_amount,
    withdraw_precompile=XTZ_WITHDRAWAL_PRECOMPILE,
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL,
)

Making Fast Withdrawal, XTZ:
  - Sender: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `https://node.ghostnet.etherlink.com`
  - Withdrawal params:
      * Target: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
      * Fast Withdrawal contract: `KT1TbrmAD4WRVqakbqrHDZ7gUwMdjocXc7Cw`
      * Payload bytes: `0500a88c01`
      * Amount (mutez): `9_119`
      * Discounted amt (mutez): `9_000`
      * Fee (mutez): `119`
Successfully initiated XTZ Fast Withdrawal, tx hash: `0xcd6bdd6eeb09df6a7ea4da31389ced1637c31b5ff13f32e19e8e8f3aff8dae21`


### Searching for outbox message:
- TODO: Move this code to a separate script

In [4]:
from scripts.helpers.rollup_node import get_messages
import time
from pytezos import MichelsonType
from pytezos import michelson_to_micheline
from scripts.etherlink.xtz_fast_withdraw import (
    load_withdraw_precompile,
    make_fast_withdrawal,
)
from random import randint
from scripts.helpers.contracts.fast_withdrawal import Withdrawal
from scripts.helpers.utility import pack
from scripts.helpers.ticket_content import TicketContent

def make_event_logs_link(tx_hash):
    return f'{BLOCKSCOUT_EXPLORER_URL}/tx/{tx_hash}?tab=logs'

def make_outbox_message_link(level):
    return f'{ETHERLINK_ROLLUP_NODE_URL}/global/block/{level}/outbox/{level}/messages'

def make_tzkt_ghostnet_link(tx_hash):
    return f'{TZKT_EXPLORER_URL}/{tx_hash}'

FAST_WITHDRAWAL_OUTBOX_INTERFACE = '''
pair %default (nat %withdrawal_id)
    (pair (ticket %ticket (pair nat (option bytes)))
         (pair (timestamp %timestamp)
               (pair (address %base_withdrawer)
                     (pair (bytes %payload) (bytes %l2_caller)))))
'''

def decode_outbox_message(message):
    micheline_expression = michelson_to_micheline(FAST_WITHDRAWAL_OUTBOX_INTERFACE)
    michelson_type = MichelsonType.match(micheline_expression)
    assert len(message['transactions']) == 1
    parameters_micheline = message['transactions'][0]['parameters']
    return michelson_type.from_micheline_value(parameters_micheline).to_python_object()

def scan_outbox_until_message_found(
    etherlink_rollup_node_url: str,
    last_level: int,
    max_levels: int = 100,
    sleep_time: int = 0.3
):
    click.echo('Scanning outbox: ', nl=False)
    for level in range(last_level, last_level-max_levels, -1):
        click.echo(wrap(accent(level)), nl=False)
        messages = get_messages(ETHERLINK_ROLLUP_NODE_URL, level)
        if len(messages) > 0:
            click.echo()
            click.echo('Found outbox message at level: ' + wrap(accent(level)))
            return messages, level
        click.echo(', ', nl=False)
        time.sleep(sleep_time)

def make_withdrawal_info(tx_hash, outbox_message, found_level: int):
    click.echo('Fast Withdrawal info:')
    click.echo('- Event logs: ' + make_event_logs_link(tx_hash))
    click.echo('- Outbox message: ' + make_outbox_message_link(found_level))
    click.echo('- Withdrawal ID: ' + wrap(accent(outbox_message['withdrawal_id'])))
    click.echo('- Timestamp: ' + wrap(accent(outbox_message['timestamp'])))

In [5]:
last_tezos_level = tezos_account.shell.head()['header']['level']
messages, found_level = scan_outbox_until_message_found(ETHERLINK_ROLLUP_NODE_URL, last_tezos_level)
outbox_message = decode_outbox_message(messages[0]['message'])
make_withdrawal_info(tx_hash, outbox_message, found_level)

Scanning outbox: `11773200`
Found outbox message at level: `11773200`
Fast Withdrawal info:
- Event logs: https://testnet.explorer.etherlink.com/tx/0xcd6bdd6eeb09df6a7ea4da31389ced1637c31b5ff13f32e19e8e8f3aff8dae21?tab=logs
- Outbox message: https://ghostnet-smart.tzkt.io//global/block/11773200/outbox/11773200/messages
- Withdrawal ID: `2055`
- Timestamp: `1744449172`


## Purchasing withdrawal on Tezos side:

In [6]:
withdrawal = Withdrawal(
    withdrawal_id=outbox_message['withdrawal_id'],
    full_amount=full_amount,
    ticketer=XTZ_TICKETER_ADDRESS,
    content=TicketContent(
        token_id=0,
        token_info=None,
    ),
    timestamp=outbox_message['timestamp'],
    base_withdrawer=get_address(tezos_account),
    payload=pack(discounted_amount, 'nat'),
    l2_caller=bytes.fromhex(etherlink_account.address.split('0x')[1])
)

opg = fast_withdrawal.payout_withdrawal(
    withdrawal=withdrawal,
    service_provider=get_address(tezos_account),
    xtz_amount=discounted_amount,
).send()

click.echo('- Transaction: ' + make_tzkt_ghostnet_link(opg.hash()))

- Transaction: https://quebecnet.tzkt.io/ooHDBWYVcNqHSJyFDR55gzUVJ8PMQDBFCT2428FMA6MSKUDuznJ


## Making Additional Withdrawals:
### One with an incorrect payload that would be impossible to execute

In [7]:
precompile_contract = load_withdraw_precompile(
    XTZ_WITHDRAWAL_PRECOMPILE,
    web3
)
receipt = make_fast_withdrawal(
    etherlink_account=etherlink_account,
    web3=web3,
    precompile_contract=precompile_contract,
    target=get_address(tezos_account),
    fast_withdrawals_contract=fast_withdrawal.address,
    payload=bytes.fromhex('ff4f'),
    wei_amount=133 * 10**12,
)

click.echo(
    'Successfully initiated wrong XTZ Fast Withdrawal, tx hash: '
    + wrap(accent(receipt.transactionHash.hex()))
)

Successfully initiated wrong XTZ Fast Withdrawal, tx hash: `0xdcc3dc675f54e5dfa007ce7ebac7c85c9ee004b34c18038f609047eebaad52ed`


### More Fast Withdrawals with different base withdrawers and amounts

In [8]:
IS_MAKING_PAYOUTS = True
WITHDRAWALS_COUNT = 3
SLEEP_TIME_MINUTES = 1

for num in range(1, 1 + WITHDRAWALS_COUNT):
    click.echo('--- W I T H D R A W A L   :   ' + accent(num))

    full_amount = randint(100, 1000)
    full_amount_wei = full_amount * 10**12
    discounted_amount = int(full_amount * 0.95)
    random_target = tezos_account.key.generate(export=False).public_key_hash()

    tx_hash = xtz_fast_withdraw.callback(
        target=random_target,
        fast_withdrawal_contract=fast_withdrawal.address,
        amount=full_amount_wei,
        discounted_amount=discounted_amount,
        withdraw_precompile=XTZ_WITHDRAWAL_PRECOMPILE,
        etherlink_private_key=ETHERLINK_PRIVATE_KEY,
        etherlink_rpc_url=ETHERLINK_RPC_URL,
    )

    time.sleep(30)
    last_tezos_level = tezos_account.shell.head()['header']['level']
    messages = scan_outbox_until_message_found(ETHERLINK_ROLLUP_NODE_URL, last_tezos_level)
    if not 'message' in messages[0]:
        click.echo('! Error: ' + wrap(accent(messages[0])))
        # TODO: make repeat until succeed wrapper
        continue

    outbox_message = decode_outbox_message(messages[0]['message'])
    make_withdrawal_info(tx_hash, outbox_message)

    # TODO: `Withdrawal.from_outbox`?
    withdrawal = Withdrawal(
        withdrawal_id=outbox_message['withdrawal_id'],
        full_amount=full_amount,
        ticketer=XTZ_TICKETER_ADDRESS,
        content=TicketContent(
            token_id=0,
            token_info=None,
        ),
        timestamp=outbox_message['timestamp'],
        base_withdrawer=random_target,
        payload=pack(discounted_amount, 'nat'),
        l2_caller=bytes.fromhex(etherlink_account.address.split('0x')[1])
    )

    if IS_MAKING_PAYOUTS:
        opg = fast_withdrawal.payout_withdrawal(
            withdrawal=withdrawal,
            service_provider=get_address(tezos_account),
            xtz_amount=discounted_amount,
        ).send()

        click.echo('- Payout transaction: ' + make_tzkt_ghostnet_link(opg.hash()))

    time.sleep(SLEEP_TIME_MINUTES * randint(45, 75))
    click.echo('')

--- W I T H D R A W A L   :   1
Making Fast Withdrawal, XTZ:
  - Sender: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `https://node.ghostnet.etherlink.com`
  - Withdrawal params:
      * Target: `tz1f2BNQry3w9yERDwZKtGtAMBmLqdsB2fAT`
      * Fast Withdrawal contract: `KT1TbrmAD4WRVqakbqrHDZ7gUwMdjocXc7Cw`
      * Payload bytes: `0500a60d`
      * Amount (mutez): `916`
      * Discounted amt (mutez): `870`
      * Fee (mutez): `46`
Successfully initiated XTZ Fast Withdrawal, tx hash: `0xf40f93e189389c7d1bc28d2d785556210495eac8584acea2e234c870a89b9211`
Scanning outbox: `11773226`, `11773225`, `11773224`, `11773223`, `11773222`, `11773221`
Found outbox message at level: `11773221`
! Error: `[{'outbox_level': 11773221, 'message_index': '0', 'message': {'transactions': [{'parameters': {'prim': 'Pair', 'args': [{'int': '2057'}, {'prim': 'Pair', 'args': [{'prim': 'Pair', 'args': [{'bytes': '012370218d43c505b233b950377fedd8283cec527a00'}, {'prim': 'Pair', 'args': [{'pri

## Executing outbox message:

In [20]:
from scripts.helpers.rollup_node import get_proof
from scripts.tezos.execute_outbox_message import execute_outbox_message

proof = get_proof(ETHERLINK_ROLLUP_NODE_URL, 3153834, 0)
opg = execute_outbox_message.callback(
    commitment=proof['commitment'],
    proof=proof['proof'],
    smart_rollup_address=SMART_ROLLUP_ADDRESS,
    tezos_private_key=TEZOS_PRIVATE_KEY,
    tezos_rpc_url=TEZOS_RPC_URL,
)

Executing outbox message:
  - Commitment: `src12pwFZr2zZwGzPCS2BJo41Gn4o6E8kvEciavKshU46mvLiduFA2`
  - Proof: `03000224345f5a1ac5ca3cc184769348f920591b89a070fd8c4c4795e6d79a46da466e24345f5a1ac5ca3cc184769348f920591b89a070fd8c4c4795e6d79a46da466e0005820764757261626c65d0e1876fff35c4574054bdbeeb9f9f908874e9eeedaa0bf0340acc72ed4a51542703746167c00800000004536f6d650003c0ac84336b4de3ae7d64d6aa244c5045b94fa45b05f8cbe2b181c970c489054493820576616c7565810370766d8107627566666572738205696e707574820468656164c00100066c656e677468c00100066f75747075740004820132810a6c6173745f6c6576656cc004003020020133810f76616c69646974795f706572696f64c0040003b10082013181086f7574626f78657301011700940052c0daa0a9689109496a62bdad5aa64ceca9d41220839c7c998dab6dd0297a0f2cd2002d0019c06aa55abd97ddde3130540ce49ec98b60c372880b0142433c09814477297c08bd000d00060003820733313533373335820468656164c00100066c656e677468c0010007333135333833340003810468656164c001008208636f6e74656e7473810130c0a10000009d000000009807070000070707070a0000001601bf7